In [4]:
from tensorflow import keras
import torch
from pymongo import MongoClient
import numpy as np
import pickle
from bson.binary import Binary
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

client=MongoClient('133.2.210.24',27017) #ip, port
db=client["review"] #database name
collection=db["test_vector2"] #collection name
#query={} #if you want to use only some data in database, use query to find them
cursor=collection.find({},{"_id":0,"id":1,"vector":1,"label":1})
train_data=np.array([])
train_label=np.array([])
temp_data=[]
for cur in cursor:
    dic=cur
    temp_data.append(pickle.loads(dic['vector']))
    train_label=np.append(train_label,np.array(dic['label']))
train_data=np.array(temp_data)

n_train = int(len(temp_data)*0.9)
for now in range(n_train):
    d_train,d_test = np.split(train_data, [n_train])  # 学習データを訓練用とテスト用に分割
    l_train,l_test = np.split(train_label, [n_train])  # ラベルデータを訓練用とテスト用に分割


model = Sequential()
model.add(Dense(1536, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

y_train = keras.utils.to_categorical(l_train)
y_test = keras.utils.to_categorical(l_test)

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy']) ### This is where you use the loss function.

model.fit(d_train, y_train, epochs=10, batch_size=128)
score = model.evaluate(d_test, y_test, batch_size=128)
print(score)

Epoch 1/10
528/528 [==============================] - 4s 6ms/step - loss: 0.4929 - accuracy: 0.7613
Epoch 2/10
528/528 [==============================] - 3s 6ms/step - loss: 0.4025 - accuracy: 0.8227
Epoch 3/10
528/528 [==============================] - 3s 6ms/step - loss: 0.3669 - accuracy: 0.8410
Epoch 4/10
528/528 [==============================] - 3s 6ms/step - loss: 0.3429 - accuracy: 0.8542
Epoch 5/10
528/528 [==============================] - 3s 7ms/step - loss: 0.3243 - accuracy: 0.8636
Epoch 6/10
528/528 [==============================] - 3s 6ms/step - loss: 0.3080 - accuracy: 0.8702
Epoch 7/10
528/528 [==============================] - 3s 6ms/step - loss: 0.2892 - accuracy: 0.8795
Epoch 8/10
528/528 [==============================] - 3s 6ms/step - loss: 0.2726 - accuracy: 0.8861
Epoch 9/10
528/528 [==============================] - 3s 6ms/step - loss: 0.2606 - accuracy: 0.8915
Epoch 10/10
59/59 [==============================] - 0s 2ms/step - loss: 0.3330 - accuracy: 0.8557
[

In [5]:
model.save("model_binary_keras.h5")

In [7]:
predicted=model.predict(np.expand_dims(d_train[0], 0))
print(predicted)

[[0.9107604  0.08923956]]
